In [39]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np

from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import accuracy_score, f1_score, make_scorer, recall_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_curve, recall_score, accuracy_score, precision_score, plot_confusion_matrix, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [47]:
#Importar datasets
df = pd.read_csv("../preprocessing/data_p4/trainModel_2.csv")
test = pd.read_csv("../preprocessing/data_p4/testModel_2.csv")
test_kaggle = pd.read_csv("../preprocessing/data_p3/test.csv")

In [48]:
def classificacio(clf, tuning=False):
  clf.fit(df.iloc[:,:-1], df.iloc[:,-1])
  preds = pd.Series(clf.predict(test.iloc[:,:-1]), name='preds')
  reals = test.iloc[:,-1]
  reals.name = 'reals'
  reals.index = range(len(preds))
  print('acc: ', accuracy_score(reals, preds))
  print('rec: ', recall_score(reals,preds, pos_label=1))
  print('prec: ',recall_score(reals,preds, pos_label=1))
  print('f1:', f1_score(reals, preds, pos_label=1))
  if tuning:
    print('best pars:', clf.best_params_)
    pd.crosstab(reals,preds)
  return [reals,preds,clf]


In [49]:
resultats1={}

In [50]:
resultats1['RandomForestClassifier']=classificacio(GradientBoostingClassifier())
pd.crosstab(resultats1['RandomForestClassifier'][0],resultats1['RandomForestClassifier'][1])

acc:  0.7922208610430521
rec:  0.5883424408014571
prec:  0.5883424408014571
f1: 0.5763225979123918


preds,0,1
reals,,
0,14877,2489
1,2260,3230


In [51]:
results_kaggle = resultats1['RandomForestClassifier'][2].predict(test_kaggle)
results_final_kaggle = pd.DataFrame()
test_kaggle = pd.read_csv("../preprocessing/Raw data/test.csv", sep =';')
results_final_kaggle['order_id']  = test_kaggle['order_id'] 
results_final_kaggle['late_order'] = results_kaggle


In [52]:
results_final_kaggle.to_csv('results_kaggle_2.csv', index=False)
